In [51]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares



In [3]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [4]:
df = pd.read_csv('data/pp-contingency-matrix.csv')
df

,uniprot_id,reactome_pathway_id
0,A0A075B5J3,R-MMU-198955
1,A0A075B5J3,R-MMU-202165
2,A0A075B5J3,R-MMU-202168
3,A0A075B5J3,R-MMU-202174
4,A0A075B5J3,R-MMU-202214
...,...,...
79492,V9GXR0,R-MMU-983422
79493,V9GXR0,R-MMU-983424
79494,V9GXR0,R-MMU-983425
79495,V9GXR0,R-MMU-983427


In [5]:
protein_pathway_df = pd.crosstab(df['uniprot_id'], df['reactome_pathway_id'])
protein_pathway_df

reactome_pathway_id,R-MMU-1006143,R-MMU-1006169,R-MMU-1008200,R-MMU-1008243,R-MMU-1008248,R-MMU-1011576,R-MMU-1011598,R-MMU-1011600,R-MMU-1013012,R-MMU-1013013,...,R-MMU-994140,R-MMU-994148,R-MMU-996727,R-MMU-996755,R-MMU-997237,R-MMU-997263,R-MMU-997309,R-MMU-997311,R-MMU-997314,R-MMU-997326
uniprot_id,,,,,,,,,,,,,,,,,,,,,
A0A075B5J3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A075B5J4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A075B5J7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A075B5J9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A075B5K0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V9GWY0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
V9GX35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
V9GXQ3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# compute the number of 1 in each row
stats = protein_pathway_df.sum(axis=1)
# extract the average, minimum and maximum from stats
avg = stats.mean()
min = stats.min()
max = stats.max()
# print the results
print('Average: ', avg)
print('Minimum: ', min)
print('Maximum: ', max)
# compute the frequency of each value in stats
freq = stats.value_counts()
# print the frequency
print(freq)


Average:  8.304293324976497
Minimum:  1
Maximum:  446
1      2853
2      1378
4       885
3       820
5       524
       ... 
104       1
86        1
192       1
100       1
95        1
Name: count, Length: 112, dtype: int64


In [7]:
protein_pathway_matrix = protein_pathway_df.values
protein_pathway_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
# print the number of colmns 
print('Number of columns: ', protein_pathway_matrix.shape[1])

Number of columns:  9390


## Autoencoders

In [26]:
ae = Autoencoder(protein_pathway_matrix.shape[1], 50)
criterion = nn.BCELoss()
optimizer = optim.Adam(ae.parameters(), lr=0.001)

protein_pathway_tensor = torch.FloatTensor(protein_pathway_matrix)

for epoch in range(100):  # Number of epochs
    optimizer.zero_grad()  # Zero the gradients
    outputs = ae(protein_pathway_tensor)  # Forward pass
    loss = criterion(outputs, protein_pathway_tensor)  # Compute the loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 100, loss.item()))

Epoch [1/100], Loss: 0.6944
Epoch [2/100], Loss: 0.6937
Epoch [3/100], Loss: 0.6929
Epoch [4/100], Loss: 0.6919
Epoch [5/100], Loss: 0.6908
Epoch [6/100], Loss: 0.6893
Epoch [7/100], Loss: 0.6877
Epoch [8/100], Loss: 0.6857
Epoch [9/100], Loss: 0.6834
Epoch [10/100], Loss: 0.6808
Epoch [11/100], Loss: 0.6779
Epoch [12/100], Loss: 0.6746
Epoch [13/100], Loss: 0.6709
Epoch [14/100], Loss: 0.6669
Epoch [15/100], Loss: 0.6625
Epoch [16/100], Loss: 0.6577
Epoch [17/100], Loss: 0.6526
Epoch [18/100], Loss: 0.6471
Epoch [19/100], Loss: 0.6412
Epoch [20/100], Loss: 0.6350
Epoch [21/100], Loss: 0.6284
Epoch [22/100], Loss: 0.6215
Epoch [23/100], Loss: 0.6143
Epoch [24/100], Loss: 0.6068
Epoch [25/100], Loss: 0.5990
Epoch [26/100], Loss: 0.5910
Epoch [27/100], Loss: 0.5828
Epoch [28/100], Loss: 0.5744
Epoch [29/100], Loss: 0.5658
Epoch [30/100], Loss: 0.5571
Epoch [31/100], Loss: 0.5483
Epoch [32/100], Loss: 0.5393
Epoch [33/100], Loss: 0.5303
Epoch [34/100], Loss: 0.5212
Epoch [35/100], Loss: 0

In [27]:
z = ae.encoder(torch.from_numpy(protein_pathway_matrix).float())
z

tensor([[1.3409, 1.1902, 1.2969,  ..., 1.3034, 1.3574, 1.3377],
        [1.3409, 1.1902, 1.2969,  ..., 1.3034, 1.3574, 1.3377],
        [2.3627, 1.8935, 1.7375,  ..., 2.4602, 2.5662, 2.4860],
        ...,
        [0.8103, 0.7288, 0.7123,  ..., 0.8376, 0.7317, 0.7703],
        [1.6432, 1.5359, 1.5853,  ..., 1.7429, 1.6684, 1.7545],
        [0.2384, 0.2281, 0.2227,  ..., 0.2328, 0.2161, 0.2320]],
       grad_fn=<ReluBackward0>)

In [50]:
target_protein = torch.from_numpy(protein_pathway_matrix[60]).float()
print(target_protein)

target = ae.encoder(target_protein)
similarities = cosine_similarity([target.detach().numpy()], z.detach().numpy())

k = 5
top_k_proteins = similarities.argsort()[0][::-1][:k]
for p in top_k_proteins:
    print(p, similarities[0][p])

tensor([0., 0., 0.,  ..., 0., 0., 0.])
50 1.0000001
20 1.0000001
19 1.0000001
1511 1.0000001
43 1.0000001


## MF

In [67]:
matrix = csr_matrix(protein_pathway_matrix)
mf_model = AlternatingLeastSquares(factors=120)

mf_model.fit(matrix)

protein_factors = mf_model.user_factors

100%|██████████| 15/15 [00:00<00:00, 31.29it/s]


In [69]:
target_protein = 30
protein_factors = mf_model.user_factors
similarities = cosine_similarity([protein_factors[target_protein]], protein_factors)

k = 5
top_k_proteins = similarities.argsort()[0][::-1][:k]
for p in top_k_proteins:
    print(p, similarities[0][p])




82 1.0
100 1.0
41 1.0
40 1.0
39 1.0


## Subset selection

In [58]:
train = pd.read_csv('data/train.csv', index_col=0)
target = 'group'
features = train.columns.tolist()
features.remove(target)
features

['A0A067XG53',
 'A0A075B5P4',
 'A0A075B5P5',
 'A0A075B5P6',
 'A0A087WNV1',
 'A0A087WP47',
 'A0A087WPF0',
 'A0A087WPP8',
 'A0A087WPT7',
 'A0A087WPZ6',
 'A0A087WQM0',
 'A0A087WQN7',
 'A0A087WQS2',
 'A0A087WR42',
 'A0A087WRH2',
 'A0A087WRY3',
 'A0A087WS03',
 'A0A0A0MQ68',
 'A0A0A0MQ79',
 'A0A0A0MQ90',
 'A0A0A0MQ98',
 'A0A0A0MQ99',
 'A0A0A0MQA5',
 'A0A0A0MQB0',
 'A0A0A6YVS2',
 'A0A0A6YWB0',
 'A0A0A6YWP6',
 'A0A0A6YWS6',
 'A0A0A6YWX1',
 'A0A0A6YX02',
 'A0A0A6YX73',
 'A0A0A6YXC8',
 'A0A0A6YXL6',
 'A0A0A6YXW7',
 'A0A0A6YY47',
 'A0A0B4J1G1',
 'A0A0B4J1G5',
 'A0A0B4J1L4',
 'A0A0D2X7Z2',
 'A0A0F6AIX5',
 'A0A0G2JDI9',
 'A0A0G2JDW1',
 'A0A0G2JDW7',
 'A0A0G2JE26',
 'A0A0G2JE99',
 'A0A0G2JEC4',
 'A0A0G2JEH5',
 'A0A0G2JEI3',
 'A0A0G2JEK2',
 'A0A0G2JES4',
 'A0A0G2JF16',
 'A0A0G2JFC9',
 'A0A0G2JFF9',
 'A0A0G2JFH0',
 'A0A0G2JFJ6',
 'A0A0G2JFY0',
 'A0A0G2JFZ4',
 'A0A0G2JG52',
 'A0A0G2JG82',
 'A0A0G2JGJ1',
 'A0A0G2JGL8',
 'A0A0G2JGQ6',
 'A0A0G2JH06',
 'A0A0G2JH18',
 'A0A0J9YU62',
 'A0A0J9YUR2',
 'A0A0J9YU

In [64]:
best = []
acc = {}
while len(best) < 3:
    max_acc = 0
    remaining_features = list(set(features) - set(best))
    new_pval = pd.Series(index=remaining_features)
    for new_column in remaining_features:
        model = LogisticRegression()
        model.fit(train[best + [new_column]], train[target])
        target_predicted = model.predict(train[best + [new_column]])
        acc = metrics.accuracy_score(train[target], target_predicted)
        if max_acc < acc:
            max_acc = acc
            max_column = new_column
    best.append(max_column)
    features.remove(max_column)
    print('Best columns: ', best)
    print('Accuracy: ', max_acc)

Best columns:  ['E9QAE3']
Accuracy:  1.0
Best columns:  ['E9QAE3', 'Q8R2U0-2']
Accuracy:  1.0
Best columns:  ['E9QAE3', 'Q8R2U0-2', 'H9KV01']
Accuracy:  1.0


In [71]:
best = []
acc = {}
max_acc = 0
remaining_features = list(set(features) - set(best))
new_pval = pd.Series(index=remaining_features)
for new_column in remaining_features:
    model = LogisticRegression()
    model.fit(train[best + [new_column]], train[target])
    target_predicted = model.predict(train[best + [new_column]])
    acc = metrics.accuracy_score(train[target], target_predicted)
    if max_acc < acc:
        max_acc = acc
        max_column = new_column
best.append(max_column)
features.remove(max_column)

while len(best) < 3:
    last = best[-1]
    # get from reactome_pathway_id the index of the row with last
    target_protein = protein_pathway_df.index.get_loc(last)
    similarities = cosine_similarity([protein_factors[target_protein]], protein_factors)

    k = 5
    top_k_proteins = similarities.argsort()[0][::-1][:k]

    for k in top_k_proteins:
        max_acc = 0
        new_column = protein_pathway_df.index[k]
        print(k, new_column)
        


KeyError: 'S4R1P5'

In [74]:
key = 'A0A075B5J9'
if key in protein_pathway_df.index:
    print('Found')
    print(protein_pathway_df.loc[key])
else:
    print('Not found')


Found
reactome_pathway_id
R-MMU-1006143    0
R-MMU-1006169    0
R-MMU-1008200    0
R-MMU-1008243    0
R-MMU-1008248    0
                ..
R-MMU-997263     0
R-MMU-997309     0
R-MMU-997311     0
R-MMU-997314     0
R-MMU-997326     0
Name: A0A075B5J9, Length: 9390, dtype: int64


In [ ]:
        model = LogisticRegression()
        model.fit(train[best + [new_column]], train[target])
        target_predicted = model.predict(train[best + [new_column]])
        acc = metrics.accuracy_score(train[target], target_predicted)
        if max_acc < acc:
            max_acc = acc
            max_column = new_column

    best.append(max_column)
    features.remove(max_column)
    print('Best columns: ', best)
    print('Accuracy: ', max_acc)



In [ ]:


k = 5
top_k_proteins = similarities.argsort()[0][::-1][:k]
for p in top_k_proteins:
    print(p, similarities[0][p])   

    print('Best columns: ', best)
    print('Accuracy: ', max_acc)